In [1]:
%pip install langchain langchain_community openai tiktoken faiss-cpu

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 973.5/973.5 kB 4.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 27.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 320.6/320.6 kB 20.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 308.5/308.5 kB 15.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 122.8/122.8 kB 3.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 1.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.3/49.3 kB 2.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 4.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 4.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.0/53.0 kB 3.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 142.5/142.5 kB 4.3 MB/s eta 0:00:00
  Attempting uninstall: packaging
    Found existing installation: packaging 24.0
    Uninstalling packaging-

In [26]:
from langchain.prompts.example_selector import LengthBasedExampleSelector,MaxMarginalRelevanceExampleSelector,NGramOverlapExampleSelector,SemanticSimilarityExampleSelector

**LengthBasedExampleSelector**：对于较长的输入，会选择较少的示例；对于较短的输入，会选择更多的示例。

**MaxMarginalRelevanceExampleSelector**：根据示例与输入的相似度及示例之间的多样性来选择示例。通过找到与输入最相似的示例来迭代添加示例，同时对已选择的示例进行惩罚

**NGramOverlapExampleSelector**：根据示例与输入的n-gram重叠度选择和排序示例。

**SemanticSimilarityExampleSelector**：根据示例与输入的相似度来选择示例，通过找到与输入最相似的示例来实现。

In [4]:
from langchain.prompts.few_shot import FewShotPromptTemplate
from langchain.prompts.prompt import PromptTemplate

In [3]:
examples = [
    {"input": "高", "output": "矮"},
    {"input": "胖", "output": "瘦"},
    {"input": "精力充沛", "output": "萎靡不振"},
    {"input": "快乐", "output": "伤心"},
    {"input": "黑", "output": "白"},
]

In [9]:
example_prompt = PromptTemplate(input_variables=["input", "output"], template = """
词语：{input}\n
反义词：{output}\n
"""
)

# LengthBasedExampleSelector

In [10]:
example_selector = LengthBasedExampleSelector(examples=examples, example_prompt=example_prompt, max_length=25,)

In [11]:
example_selector_prompt = FewShotPromptTemplate(example_selector=example_selector,
                                                example_prompt=example_prompt,
                                                example_separator="\n",
                                                prefix="来玩个反义词接龙游戏，我说词语，你说它的反义词\n",
                                                suffix="现在轮到你了，词语：{input}\n反义词：",
                                                input_variables=["input"],
                                                )
example_selector_prompt.format(input="好")

'来玩个反义词接龙游戏，我说词语，你说它的反义词\n\n\n词语：高\n\n反义词：矮\n\n\n\n词语：胖\n\n反义词：瘦\n\n\n\n词语：精力充沛\n\n反义词：萎靡不振\n\n\n\n词语：快乐\n\n反义词：伤心\n\n\n现在轮到你了，词语：好\n反义词：'

# MaxMarginalRelevanceExampleSelector

In [14]:
from langchain_community.embeddings import OpenAIEmbeddings
from langchain_community.vectorstores import FAISS

In [16]:
# 设置OpenAI KEY环境变量
import os
import getpass
os.environ['OPENAI_API_KEY'] = getpass.getpass('OpenAI API Key:')

OpenAI API Key:··········


In [23]:
example_selector = MaxMarginalRelevanceExampleSelector.from_examples(examples, OpenAIEmbeddings(), FAISS, k=2)

# NGramOverlapExampleSelector

In [24]:
example_selector = NGramOverlapExampleSelector(examples=examples, example_prompt=example_prompt, threshold=-1.0)

# SemanticSimilarityExampleSelector

In [27]:
example_selector = SemanticSimilarityExampleSelector.from_examples(examples, OpenAIEmbeddings(), FAISS, k=1)